In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
from collections import defaultdict
from functools import partial

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymbd
from pymbd import MBDCalc
import seaborn as sns
import xarray as xr
from pkg_resources import resource_filename, resource_stream
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from tqdm.auto import tqdm

from mbdvv.app import app
from mbdvv.layered import get_layered_dataset, mbd_from_row, binding_per_area
from mbdvv.report import mbd_energy, vdw_params, plot_ion_alpha
from mbdvv.functasks import calc_vvpol
from mbdvv.physics import alpha_kin, ion_pot, vv_pol, lg_cutoff2, logistic, scanintp, eV

pd.options.display.max_rows = 999 
tqdm.pandas()

sns.set(
    style='ticks',
    context='notebook', rc={
        'axes.formatter.useoffset': False,
    },
)
mpl.rc('font', family='serif', serif='STIXGeneral')
mpl.rc('mathtext', fontset='stix')
warnings.filterwarnings(
    'ignore',
    'Sorting because non-concatenation axis is not aligned',
        FutureWarning
)

pymbd.stop_interactive()
mbd = pymbd.interactive()

In [ ]:
with app.context():
    layered_results = app.get('layered')

In [ ]:
layered_df, grids_df, freq_df = get_layered_dataset(layered_results)

In [ ]:
for lvl in layered_df.index.levels:
    print(lvl)

In [ ]:
layered_df.loc(0)[:, :, :, 6].iloc[0]

In [ ]:
(
    layered_df[['energy', 'energy_vdw']].to_xarray()
    .sel(xc='pbe', kz=6)
    .pipe(lambda ds: xr.concat(
        [ds['energy']-ds['energy_vdw'], ds['energy']],
        pd.Index(['PBE', 'PBE+MBD'], name='method')
    ))
    .pipe(lambda x: x-x.sel(shift=40))
    .to_dataframe('energy')
    .reset_index()
    .pipe(
        lambda x: sns.relplot(
            data=x,
            kind='line',
            x='shift',
            y='energy',
            row='method',
            hue='label',
            height=3,
            aspect=1.5,
        )
        .set(xlim=(-.4, .7), ylim=(None, 0.002))
    )
)

In [ ]:
layered_meta_df = (
    layered_df['lattice_vector']
    .loc(0)[:, 0, 'pbe', 6]
    .pipe(lambda x: x*0.5291)
    .apply(lambda x: np.linalg.det(x)/x[2, 2])
    .reset_index(['xc', 'kz', 'shift'], drop=True)
    .to_frame('area')
    .join(
        pd.read_csv(
            resource_stream('mbdvv', 'data/layered.csv'),
            index_col='label scale'.split()
        )
        .xs(1, level='scale')[['c', 'energy']]
        .rename(columns={'c': 'c_ref', 'energy': 'energy_ref'})
    )
    .assign(n_layer=lambda x: np.where((x['c_ref'] > 10) | (x['area'] < 6), 2, 1))
)
layered_meta_df.iloc[:1]

In [ ]:
(
    layered_df['energy'].to_xarray()
    .sel(xc='pbe', kz=6)
    .pipe(lambda x: x-x.sel(shift=40))
    .sel(shift=slice(None, 4))
    .to_dataframe()
    .reset_index()
    .groupby('label')
    .apply(
        lambda x: pd.Series(minimize(
            interp1d(x['shift'].values, x['energy'].values, kind='cubic'),
            [0],
            bounds=[(-.4, .7)],
        ))
    )[['fun', 'x']]
    .applymap(lambda x: x[0])
    .rename(columns={'fun': 'energy_uc', 'x': 'c_shift'})
    .join(layered_meta_df)
    .assign(energy=binding_per_area)
)

In [ ]:
(
    layered_df
    .loc(0)[:, :, 'pbe', 6]
    .assign(vv_shave=lambda x: (x['vv_pols_nm']-x['vv_pols'])/x['vv_pols'])
    .groupby('label shift'.split())
    .apply(lambda x: pd.DataFrame({
        'vv_shave': x['vv_shave'].iloc[0][:, 0],
        'vv_pols': x['vv_pols'].iloc[0][:, 0],
        'vv_pols_nm': x['vv_pols_nm'].iloc[0][:, 0],
        'elem': x['elems'].iloc[0]
    }))
    .groupby('shift').apply(lambda x: x['vv_shave'].describe())
    .loc(0)[[-.4, 0, 40]]
)

In [ ]:
layered_df.iloc(0)[3].vv_pols.shape

In [ ]:
(
    layered_df
    .loc(0)[:, [0, 40], 'pbe', 6]['vv_pols vv_pols_nm'.split()]
    .unstack('shift')
    .apply(lambda x: ((x[:, 0]-x[:, 40])/x[:, 40]).apply(lambda x: x[:, 0].mean()), axis=1)
)

In [ ]:
with MBDCalc() as mbd_full:
    mbd_enes_ref = (
        layered_df
        .loc(0)[:, 0.1, 'pbe', 6]
        .progress_apply(
            mbd_from_row,
            axis=1,
            beta=0.81,
            k_grid=(40, 40, 6),
            mbd=mbd_full,
        )
    )

In [ ]:
# Check we run the same MBD as in FHI-aims
(
    mbd_enes_ref.to_frame('ene_mbd')
    .join(layered_df[['energy_vdw']])
    .pipe(lambda x: (x['ene_mbd']-x['energy_vdw'])/x['energy_vdw'])
    .describe()
)

In [ ]:
mbd_enes = pd.concat([
    layered_df
    .loc(0)[:, :, 'pbe', 6]
    .progress_apply(
        mbd_from_row,
        beta=beta,
        k_grid=(10, 10, 2),
        axis=1,
        mbd=mbd,
    )
    .to_frame('ene_mbd')
    .assign(beta=beta)
    .set_index('beta', append=True)
    for beta in [0.79, 0.81, 0.83]
])['ene_mbd'].sort_index()

In [ ]:
# Relative error in total binding energy per area
# by doing smaller MBD k-grid
(
    mbd_enes.to_frame('ene_mbd')
    .xs(0.81, level='beta')
    .join(layered_df[['energy_vdw']])
    .to_xarray()
    .pipe(lambda x: x-x.sel(shift=40))
    .sel(kz=6, xc='pbe', shift=0)
    .pipe(lambda x: (x['ene_mbd']-x['energy_vdw']))
    .to_dataframe('energy_uc')
    .join(layered_meta_df)
    .assign(energy=binding_per_area)
    .pipe(lambda x: x['energy']/x['energy_ref'])
    .describe()
)

In [ ]:
(
    layered_df[['energy', 'energy_vdw']]
    .fillna(0)
    .pipe(lambda x: x['energy']-x['energy_vdw'])
    .to_frame('ene_scf')
    .join(
        mbd_enes
        .reset_index('xc kz'.split(), drop=True)
        .reset_index('beta'),
        on='label shift'.split()
    )
    .rename_axis('label shift xc kz'.split(), axis=0)
    .set_index('beta', append=True)
    .pipe(lambda x: x['ene_scf']+x['ene_mbd'])
    .to_xarray()
    .pipe(lambda x: x-x.sel(shift=40))
    .sel(shift=0)
    .to_dataframe('energy_uc')
    .join(layered_meta_df)
    .assign(energy=binding_per_area)
    .assign(rel_error=lambda x: (-x['energy']+x['energy_ref'])/x['energy_ref'])
    .assign(abs_rel_error=lambda x: x['rel_error'].abs())
    .loc(0)[:, 'pbe', 6, 0.81]
#     .groupby('xc kz beta'.split())
    .loc(0)[['BN', 'PbO', 'graphite']]
    .describe() 
)

In [ ]:
def mbd_from_row_2(row, **kwargs):
    vols = row['volumes']/row['volumes_free'][row['species']-1]
    species = row['elems']
    alpha_0, C6, R_vdw = pymbd.from_volumes(species, vols)
    return mbd_energy(
        row['coords'],
        alpha_0,
        C6,
        R_vdw,
        0.83,
        lattice=row['lattice_vector'],
        k_grid=(10, 10, 2),
        scs=True,
        rpa=True,
        return_nan=True,
        mbd=mbd,
        **kwargs,
    )


mbd_rsscs_enes = (
    layered_df
    .loc(0)[:, [0, 40], 'pbe', 6]
    .progress_apply(mbd_from_row_2, axis=1)
)

In [ ]:
(
    layered_df[['energy', 'energy_vdw']]
    .pipe(lambda x: x['energy']-x['energy_vdw'])
    .to_frame('ene_scf')
    .join(
        mbd_rsscs_enes
        .reset_index('xc kz'.split(), drop=True)
        .to_frame('ene_mbd'),
        on='label shift'.split()
    )
    .pipe(lambda x: x['ene_scf']+x['ene_mbd'])
    .to_xarray()
    .pipe(lambda x: x-x.sel(shift=40))
    .sel(shift=0)
    .to_dataframe('energy_uc')
    .join(layered_meta_df)
    .assign(energy=binding_per_area)
    .assign(rel_error=lambda x: (-x['energy']+x['energy_ref'])/x['energy_ref'])
    .assign(abs_rel_error=lambda x: x['rel_error'].abs())
    .loc(0)[:, 'pbe', 6]
    .loc(0)[['BN', 'PbO', 'graphite']]
    .describe()
)


In [ ]:
(
    layered_df[['energy', 'energy_vdw']]
    .fillna(0)
    .pipe(lambda x: x['energy']-x['energy_vdw'])
    .to_frame('ene_scf')
    .join(
        mbd_enes
        .reset_index('xc kz'.split(), drop=True)
        .reset_index('beta'),
        on='label shift'.split()
    )
    .rename_axis('label shift xc kz'.split(), axis=0)
    .set_index('beta', append=True)['ene_scf']
    .to_xarray()
    .pipe(lambda x: x-x.sel(shift=40))
    .sel(shift=0)
    .to_dataframe('energy_uc')
    .join(layered_meta_df)
    .assign(energy=binding_per_area)
    .assign(rel_error=lambda x: (-x['energy']+x['energy_ref'])/x['energy_ref'])
    .assign(abs_rel_error=lambda x: x['rel_error'].abs())
    .loc(0)[:, 'pbe', 6, 0.81]
#     .groupby('xc kz beta'.split())
    .describe() 
)


In [ ]:
(
    pd.read_csv(resource_stream('mbdvv', 'data/bjorkman-prb-12.csv'), header=None)
    .set_index(0)
    .assign(rel_error=lambda x: (-x[4]+x[1])/x[1])
    .assign(abs_rel_error=lambda x: x['rel_error'].abs())
    [['rel_error', 'abs_rel_error']]
    .loc(0)[['BN', 'PbO', 'Graphite']]
    .describe()
)

In [ ]:
pts_df = (
    pd.concat(
        dict(
            grids_df
            .gridfile
            .apply(lambda x: pd.read_hdf(x))
        ),
        names='label shift i_point'.split()
    )
) 

In [ ]:
_tmp = (
    pts_df['i_atom part_weight rho rho_grad_norm kin_dens'.split()]
    .set_index('i_atom', append=True) \
    .assign(kin_dens=lambda x: x.kin_dens/2)
    .loc[lambda x: x.rho > 0]
    .assign(
        alpha=lambda x: alpha_kin(x.rho, x.rho_grad_norm, x.kin_dens),
        ion_pot=lambda x: ion_pot(x.rho, x.rho_grad_norm),
        vvpol=lambda x: vv_pol(x.rho, x.rho_grad_norm),
    )
)

In [ ]:
labels =  _tmp.index.levels[0]
fig, axes = plt.subplots(len(labels), 2, figsize=(6, 80))
for ax_row, label in zip(axes, labels):
    for ax, shift in zip(ax_row, [0, 40]):
        try:
            df = _tmp.loc(0)[label, shift]
        except KeyError:
            continue
        plot_ion_alpha(ax, df)[-1]
        ax.set_title((label, shift))